In [1]:
from tqdm import tqdm
import numpy as np
from sklearn.metrics import r2_score
from matplotlib import pyplot as plt
from sklearn.neural_network import MLPRegressor    #### MLP 感知机####
from sklearn.tree import ExtraTreeRegressor        #### ExtraTree极端随机树回归####
from sklearn import tree                           #### 决策树回归####
from sklearn.ensemble import BaggingRegressor      #### Bagging回归####
from sklearn.ensemble import AdaBoostRegressor     #### Adaboost回归
from sklearn import linear_model                   #### 线性回归####
from sklearn import svm                            #### SVM回归####
from sklearn import ensemble                       #### Adaboost回归####  ####3.7GBRT回归####  ####3.5随机森林回归####
from sklearn import neighbors                      #### KNN回归####
from sklearn.preprocessing import StandardScaler
import os
import pandas as pd
from evaluate_data import *
import openpyxl
import warnings
warnings.filterwarnings("ignore")

def perfromance(y_test,y_pred):
    print('RMSE: ',RMSE1(y_test,y_pred))
    print('MAPE: ',MAPE1(y_test,y_pred))
    print("MAE: ",MAE1(y_test,y_pred))
    print('r2_score: ',r2_score(y_test, y_pred))

# Preprocessing 

In [2]:
def create_data(data, train_num, ahead_num):
    train_num = round(0.8*len(data))
    
    dataX1, dataX2 = [], []
    dataY1, dataY2 = [], []

    for i in range(train_num - ahead_num):
        # print(i)
        a = data[i:(i + ahead_num), 0]
        dataX1.append(a)
    for j in range(train_num - ahead_num, len(data) - ahead_num):
        b = data[j:(j + ahead_num), 0]
        dataX2.append(b)

    dataY1 = data[ahead_num:train_num, 0]
    dataY2 = data[train_num:, 0]
    return np.array(dataX1), np.array(dataY1), np.array(dataX2), np.array(dataY2)

In [3]:
def load_data(filename, ahead_num,n):
    dataset = pd.read_csv(filename,encoding='gbk')
    dataset = pd.DataFrame(dataset)
    dataPOV = dataset[['timeStamp','ghi']]

    dataPOV['timeStamp'] = pd.to_datetime(dataPOV['timeStamp'])
    dataPOV.set_index('timeStamp', inplace=True)
    dataPOV = dataPOV.resample(f'{n}T').mean()
    dataPOV.dropna(inplace=True)
    
    dataPOV = np.array(dataPOV).reshape(-1,1)

    N1=7000
    N2 = N1+1000
    dataAll = dataPOV[:N2,:]
    
#     global scaler
#     scaler = StandardScaler(copy=True,with_mean=True,with_std=True)
#     dataAll = scaler.fit_transform(dataAll)

    trainX, trainY, testX, testY = create_data(dataAll,N1,ahead_num)
    
    return trainX, trainY, testX, testY   #dataFll

In [68]:
filename =  "Folsom_irradiance.csv"

In [69]:
df = pd.read_csv(filename,encoding='gbk')

In [70]:
df.dropna(inplace=True)

In [71]:
df.head()

,timeStamp,ghi,air_temp,windsp,winddir
0,2014/1/2 15:22,3.08,2.9,1.76,187.8
1,2014/1/2 15:23,3.08,2.9,1.90,197.8
2,2014/1/2 15:24,3.18,2.9,1.82,199.4
3,2014/1/2 15:25,3.80,2.9,2.22,197.2
4,2014/1/2 15:26,4.00,2.9,2.30,212.3


In [72]:
df.shape

(533466, 5)

In [73]:
df = df.iloc[:50000]

In [74]:
df = df[['air_temp','windsp','winddir','ghi']]

In [75]:
# X = df[['air_temp','windsp','winddir']]
# y = df['ghi']

In [76]:
df_main = pd.concat([df,df.shift(-1),df.shift(-2),df.shift(-3),df.shift(-4)],axis=1)
df_main['Target_ghi'] = df['ghi'].shift(-5)

In [78]:
df_main.dropna(inplace=True)

In [79]:
df_main

,air_temp,windsp,winddir,ghi,air_temp,windsp,winddir,ghi,air_temp,windsp,...,ghi,air_temp,windsp,winddir,ghi,air_temp,windsp,winddir,ghi,Target_ghi
0,2.90,1.76,187.8,3.08,2.90,1.90,197.8,3.08,2.90,1.82,...,3.18,2.90,2.22,197.2,3.80,2.90,2.30,212.3,4.00,4.62
1,2.90,1.90,197.8,3.08,2.90,1.82,199.4,3.18,2.90,2.22,...,3.80,2.90,2.30,212.3,4.00,2.86,2.14,248.7,4.62,4.87
2,2.90,1.82,199.4,3.18,2.90,2.22,197.2,3.80,2.90,2.30,...,4.00,2.86,2.14,248.7,4.62,2.80,2.00,199.6,4.87,5.59
3,2.90,2.22,197.2,3.80,2.90,2.30,212.3,4.00,2.86,2.14,...,4.62,2.80,2.00,199.6,4.87,2.70,1.74,190.4,5.59,1.23
4,2.90,2.30,212.3,4.00,2.86,2.14,248.7,4.62,2.80,2.00,...,4.87,2.70,1.74,190.4,5.59,2.70,1.78,193.6,1.23,1.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49991,11.00,1.22,142.9,230.80,11.00,0.70,165.4,231.00,11.08,0.88,...,229.80,11.18,1.04,169.1,227.90,11.20,1.74,170.0,225.20,223.20
49992,11.00,0.70,165.4,231.00,11.08,0.88,147.8,229.80,11.18,1.04,...,227.90,11.20,1.74,170.0,225.20,11.20,1.74,127.8,223.20,221.50
49993,11.08,0.88,147.8,229.80,11.18,1.04,169.1,227.90,11.20,1.74,...,225.20,11.20,1.74,127.8,223.20,11.20,1.12,146.3,221.50,219.60
49994,11.18,1.04,169.1,227.90,11.20,1.74,170.0,225.20,11.20,1.74,...,223.20,11.20,1.12,146.3,221.50,11.20,0.76,145.3,219.60,216.80


In [80]:
X = df_main.drop('Target_ghi',axis=1).values
y = df_main['Target_ghi'].values

In [81]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [12]:
# n = 1
# ahead_num = 8
# X_train, y_train, X_test, y_test = load_data(filename, ahead_num,n)

# Scaling

In [13]:
# df = pd.read_csv(filename,encoding='gbk')

In [82]:
from sklearn.preprocessing import MinMaxScaler
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Model SVR

In [83]:
from sklearn.svm import SVR
regressor = SVR()
regressor.fit(X_train, y_train)

SVR()

In [84]:
y_pred = regressor.predict(X_test)

In [85]:
y_pred

array([106.14849958,  21.91702265, 522.80375261, ..., 175.66089244,
        76.26679039,  61.70190755])

In [ ]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(X_train, y_train)

In [89]:
from sklearn.ensemble import RandomForestRegressor
regressorRF = RandomForestRegressor(n_estimators = 300, random_state = 0)
regressorRF.fit(X_train, y_train)

RandomForestRegressor(n_estimators=300, random_state=0)

In [90]:
y_predRF = regressorRF.predict(X_test)

# Check the Performance

In [86]:
d = pd.DataFrame({'Y_True':y_test,'Y_Pred':y_pred})
d['mape'] =  abs(d['Y_True'] - d['Y_Pred'])*100/d['Y_True']
d

,Y_True,Y_Pred,mape
0,81.50,106.148500,30.243558
1,13.44,21.917023,63.073085
2,522.00,522.803753,0.153976
3,91.70,93.710873,2.192882
4,270.40,261.475559,3.300459
...,...,...,...
9994,133.80,102.691664,23.249877
9995,564.00,534.612998,5.210461
9996,182.70,175.660892,3.852823
9997,64.75,76.266790,17.786549


In [91]:
MAPE1(y_test,y_predRF)

5.003675309753071

In [92]:
perfromance(y_test,y_predRF)

RMSE:  35.88828820423479
MAPE:  5.003675309753071
MAE:  10.577455842250888
r2_score:  0.9784464116337578


In [ ]:
RMSE:  46.87575755853037
MAPE:  79.19518879269995
MAE:  22.591594714909693
r2_score:  0.9632285550981166

# Grid Search

In [ ]:
# parameters = {
#     "kernel": ["rbf",'linear'], 
#     "gamma": ['scale', 'auto'], 
#     "C": 10.0 ** np.arange(-4, 4),
# }

parameters = {'C': [1, 10, 12], #14, 16, 18, 20, 22],
            'gamma': [0.001, 0.01, 0.1, 1, 2, 5],
            'epsilon': [0.001, 0.01, 0.1, 1, 2, 4],
            'kernel': ["rbf", "poly"]}# , "sigmoid"]}

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

def my_mape(y_true,y_pred):
    return MAPE1(y_true,y_pred)

my_scorer = make_scorer(my_mape, greater_is_better=False)

In [ ]:
# neg_mean_absolute_percentage_error
clf = GridSearchCV(SVR(),cv=5,param_grid = parameters,n_jobs=-1,
                   scoring='neg_mean_absolute_percentage_error',
                   verbose=2,return_train_score=True)

clf.fit(X_train, y_train)

In [ ]:
print(clf.best_params_)

In [ ]:
regressor_grid = SVR(C= 1000, gamma='scale', kernel= 'rbf')
regressor_grid.fit(X_train, y_train)

In [ ]:
y_pred_grid = regressor.predict(X_test)

In [ ]:
perfromance(y_test,y_pred_grid)

In [ ]:
RMSE:  12.231169329924942
MAPE:  15.25695302277054
MAE:  9.060598284431556
r2_score:  0.9944672287759391

# XgBoost

In [93]:
import xgboost as xg

# Instantiation
xgb_r = xg.XGBRegressor()

# Fitting the model
xgb_r.fit(X_train, y_train)

# Predict the model
pred = xgb_r.predict(X_test)

perfromance(y_test,pred)

RMSE:  36.65087095001673
MAPE:  9.251561486167653
MAE:  11.605975677739515
r2_score:  0.9775207045472324
